# Explore here

In [ ]:
# Your code here

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from pickle import dump

# PASO 1: Carga del conjunto de datos

url_reviews = "https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv"
df_playstore = pd.read_csv(url_reviews)


# PASO 2: Preprocesamiento de Texto


def limpiar_texto(texto):
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    # Convertir a minúsculas y quitar espacios extra
    return texto.strip().lower()

# Eliminamos la columna innecesaria y limpiamos las reseñas
df_sentimientos = df_playstore.drop("package_name", axis=1)
df_sentimientos["review"] = df_sentimientos["review"].apply(limpiar_texto)

# Separación en características (X) y objetivo (y)
X_texto = df_sentimientos["review"]
y_objetivo = df_sentimientos["polarity"]

# División en entrenamiento y prueba
X_entreno_raw, X_prueba_raw, y_entreno, y_prueba = train_test_split(
    X_texto, y_objetivo, test_size=0.2, random_state=42
)

# Transformación de texto a vectores numéricos (Bolsa de Palabras)
vectorizador = CountVectorizer(stop_words="english")
X_entreno = vectorizador.fit_transform(X_entreno_raw).toarray()
X_prueba = vectorizador.transform(X_prueba_raw).toarray()


# PASO 3: Construcción del Modelo Naive Bayes


# Seleccionamos MultinomialNB ya que es el estándar para conteo de palabras
modelo_multinomial = MultinomialNB()
modelo_multinomial.fit(X_entreno, y_entreno)

predicciones_iniciales = modelo_multinomial.predict(X_prueba)
print(f"Precisión inicial (Multinomial): {accuracy_score(y_prueba, predicciones_iniciales):.4f}")

# Comparativa rápida con otras variantes
for variante in [GaussianNB(), BernoulliNB()]:
    variante.fit(X_entreno, y_entreno)
    acc = accuracy_score(y_prueba, variante.predict(X_prueba))
    print(f"Precisión con {type(variante).__name__}: {acc:.4f}")


# PASO 4: Optimización del Modelo


# Definimos el espacio de búsqueda para los hiperparámetros
espacio_parametros = {
    "alpha": np.linspace(0.01, 10.0, 100),
    "fit_prior": [True, False]
}

# Búsqueda aleatoria para encontrar la mejor combinación
busqueda_optima = RandomizedSearchCV(
    MultinomialNB(), 
    param_distributions=espacio_parametros, 
    n_iter=50, 
    scoring="accuracy", 
    cv=5, 
    random_state=42
)

busqueda_optima.fit(X_entreno, y_entreno)
print(f"Mejores parámetros: {busqueda_optima.best_params_}")

# Entrenamos el modelo final con los mejores resultados
nb_final = busqueda_optima.best_estimator_
y_pred_final = nb_final.predict(X_prueba)

print("\n--- Informe Final de Clasificación ---")
print(classification_report(y_prueba, y_pred_final))

# PASO 5: Guardado del Modelo

with open("../models/sentimientos_naive_bayes.sav", "wb") as f:
    dump(nb_final, f)

Precisión inicial (Multinomial): 0.7933
Precisión con GaussianNB: 0.7933
Precisión con BernoulliNB: 0.7542
Mejores parámetros: {'fit_prior': True, 'alpha': np.float64(1.5236363636363637)}

--- Informe Final de Clasificación ---
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       126
           1       0.69      0.58      0.63        53

    accuracy                           0.80       179
   macro avg       0.76      0.74      0.75       179
weighted avg       0.79      0.80      0.79       179

